In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# 指定支持中文的字体，例如 'SimHei'
plt.rcParams['font.sans-serif'] = ['SimHei']
# 解决负号 '-' 显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False

In [9]:
# 加载数据集
df = pd.read_csv('system_log.csv')

# 数据预处理
# 将时间戳转换为自第一个时间戳以来的总秒数
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['timestamp'] = (df['timestamp'] - df['timestamp'].min()).dt.total_seconds()

# 对 'label' 列进行编码
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])
print(df['label'].value_counts())
num_classes = df['label'].nunique()
# 分离特征 (X) 和目标 (y)
X = df.drop(['label','timestamp'], axis=1)
y = df['label']
X.info()
# 将数据拆分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

label
1    1191
3     562
2     534
0     421
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2708 entries, 0 to 2707
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   mouse_distance     2708 non-null   float64
 1   mouse_left_click   2708 non-null   int64  
 2   mouse_right_click  2708 non-null   int64  
 3   mouse_scroll       2708 non-null   int64  
 4   keyboard_counts    2708 non-null   int64  
 5   cpu_percent        2708 non-null   float64
 6   ram_percent        2708 non-null   float64
 7   gpu_percent        2708 non-null   int64  
 8   gpu_vram_percent   2708 non-null   float64
dtypes: float64(4), int64(5)
memory usage: 190.5 KB


In [10]:
model = XGBClassifier(
    objective='multi:softprob',  # 明确指定为多分类任务
    num_class=num_classes,       # 明确告知类别的数量
    eval_metric='mlogloss'       
)
# 评估方法一: 5折交叉验证
print("--- 1. 执行5折交叉验证 ---")
# 使用 cross_val_score 函数进行交叉验证，cv=5 表示5折
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print(f"交叉验证的每次准确率分数: {cv_scores}")
print(f"交叉验证的平均准确率: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})\n")
# 在训练集上训练模型，用于后续的评估
model.fit(X_train, y_train)

# 在测试集上进行预测
y_pred = model.predict(X_test)

--- 1. 执行5折交叉验证 ---
交叉验证的每次准确率分数: [0.9797048  1.         0.99077491 0.99630314 0.94454713]
交叉验证的平均准确率: 0.9823 (+/- 0.0201)



In [11]:
# 评估方法二: 分类报告
print("--- 2. 分类报告 ---")
# 获取原始的类别名称，用于报告显示
target_names = label_encoder.classes_
# 打印每个类别的精确率、召回率和F1分数
print(classification_report(y_test, y_pred, target_names=target_names))

--- 2. 分类报告 ---
              precision    recall  f1-score   support

      coding       0.98      0.98      0.98        89
      gaming       1.00      1.00      1.00       240
        idle       1.00      1.00      1.00       109
       video       0.98      0.97      0.98       104

    accuracy                           0.99       542
   macro avg       0.99      0.99      0.99       542
weighted avg       0.99      0.99      0.99       542



In [12]:
# 评估方法三: 混淆矩阵可视化
print("--- 3. 生成混淆矩阵图 (confusion_matrix.png) ---")
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
plt.title('混淆矩阵 (Confusion Matrix)', fontsize=16)
plt.ylabel('真实标签 (Actual Label)', fontsize=12)
plt.xlabel('预测标签 (Predicted Label)', fontsize=12)
# 保存图像到文件
plt.savefig('confusion_matrix.png')
plt.close() # 关闭图像，防止显示混乱
print("混淆矩阵图已保存。\n")

--- 3. 生成混淆矩阵图 (confusion_matrix.png) ---
混淆矩阵图已保存。



In [ ]:
# 评估方法四: 特征重要性可视化
print("--- 4. 生成特征重要性图 (feature_importance.png) ---")
feature_importances = model.feature_importances_
features = X.columns
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})
# 按重要性得分降序排列
importance_df = importance_df.sort_values(by='Importance', ascending=False)
plt.figure(figsize=(12, 6))
sns.barplot(x='Importance', y='Feature', data=importance_df, palette='viridis')
plt.title('特征重要性 (Feature Importance)', fontsize=16)
plt.xlabel('重要性得分 (Importance Score)', fontsize=12)
plt.ylabel('特征 (Features)', fontsize=12)
plt.tight_layout() # 调整布局以防标签重叠
# 保存图像到文件
plt.savefig('feature_importance.png')
plt.close() # 关闭图像
print("特征重要性图已保存。\n")

print("所有评估已完成。")

--- 4. 生成特征重要性图 (feature_importance.png) ---
